# Домашнее задание Продвинутый pandas 2
## задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [300]:
import pandas as pd
data=pd.read_csv('ml-latest-small/ratings.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [332]:
data['time']=pd.to_datetime(data['timestamp'], unit='s')
days=(data.groupby(by='userId').
   agg(rating_count = ('rating','count'), 
       dif_time= ('time', lambda x: max(x)-min(x) )
      )
   .query('rating_count>100')
  .dif_time.astype('timedelta64[D]').mean()  #можно ли astype запихнуть   в lambda функцию выше?
  )

days   

463.5232558139535

## К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [221]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [222]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [223]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [224]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [249]:
sales=auto.merge(rzd, how='outer',on='client_id').merge(air,how='outer',on='client_id')
sales

,client_id,auto_revenue,rzd_revenue,air_revenue
0,113,57483.0,10283.0,NaN
1,114,83.0,5774.0,NaN
2,115,912.0,981.0,81.0
3,116,4834.0,NaN,4.0
4,117,98.0,NaN,13.0
5,111,NaN,1093.0,NaN
6,112,NaN,2810.0,NaN
7,118,NaN,NaN,173.0


In [250]:
len(sales)==len(client_base)   #проверка

True

In [246]:
client_base.merge(auto,how='left',on='client_id').merge(rzd, how='left',on='client_id').merge(air,how='left',on='client_id')

,client_id,address,auto_revenue,rzd_revenue,air_revenue
0,111,Комсомольская 4,NaN,1093.0,NaN
1,112,Энтузиастов 8а,NaN,2810.0,NaN
2,113,Левобережная 1а,57483.0,10283.0,NaN
3,114,Мира 14,83.0,5774.0,NaN
4,115,ЗЖБИиДК 1,912.0,981.0,81.0
5,116,Строителей 18,4834.0,NaN,4.0
6,117,Панфиловская 33,98.0,NaN,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## задание 3


В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
к таблице визитов присоединила бы таблицу данных пользователей merge (how='left', on='user_id'),
т.к. в выгрузке по визитам и покупкам за определенный период не все покупатели,
предварительно: 
*убрать дубликаты из табилцы пользователей  drop_duplicates(keep='last'-если мне нужны последние данные
по нахождению пользователя,например, для промо акций по регинам, 'keep'='first'-если быди специфические кампании
по продвижению по регионам, нужно понять "точку первого контакта").
**Если данные визит,покупки находятся в одном столбце(как действие),необходимо сделать pivot_table
                                        
на основе координат можно постоить карту в BI  или найти таблицу данных координат и названия городов России,
присоединить  к нашим данным и сделать рапределение по городам здесь, посчитать CR  и т.д.